Here I develop code necessary to train a model to classify geo-wiki drivers based on S2 images

In [18]:
import torch
import sklearn
from sklearn import preprocessing
import pandas as pd
import geopandas as gpd

import os
import pandas as pd
from torchvision.io import read_image

from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

Preprocessing

In [16]:
classes = filtered_controls.answer.unique()

le = preprocessing.LabelEncoder()
le.fit(classes)

filtered_controls['label'] = le.transform(filtered_controls.answer)
#list(le.inverse_transform([2, 2, 1]))

/Users/janpisl/miniconda3/envs/pytorch/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [14]:
def process_annotations_file(path):
        
        annotations_df = pd.read_csv(path)
        annotations_gdf = gpd.GeoDataFrame(annotations_df,
                            geometry=gpd.points_from_xy(annotations_df.x, annotations_df.y))    
                            
        annotations = annotations_gdf.loc[annotations_gdf.step == 'step1'].copy()

        classes = annotations.answer.unique()

        le = preprocessing.LabelEncoder()
        le.fit(classes)

        annotations['label'] = le.transform(annotations.answer)

        return annotations

class Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):

        self.img_labels = process_annotations_file(annotations_file)

        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

ValueError: y contains previously unseen labels: 'tokdyo'